In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Tiny GPT model
class TinyGPT(nn.Module):
    def __init__(self, vocab_size=100, dim=64, n_layers=2, n_heads=2):
        super().__init__()
        self.tok_emb = nn.Embedding(vocab_size, dim)
        self.pos_emb = nn.Embedding(128, dim)  # max seq len 128
        
        self.blocks = nn.ModuleList([
            nn.TransformerEncoderLayer(
                d_model=dim, 
                nhead=n_heads, 
                dim_feedforward=dim*4,
                batch_first=True,
                norm_first=True  # Pre-LN like GPT
            ) for _ in range(n_layers)
        ])
        
        self.ln_f = nn.LayerNorm(dim)
        self.head = nn.Linear(dim, vocab_size, bias=False)
    
    def forward(self, x):
        B, T = x.shape
        
        # Embeddings
        tok_emb = self.tok_emb(x)  # [B, T, dim]
        pos_emb = self.pos_emb(torch.arange(T, device=x.device))  # [T, dim]
        x = tok_emb + pos_emb
        
        # Transformer blocks (with causal mask)
        causal_mask = nn.Transformer.generate_square_subsequent_mask(T, device=x.device)
        for block in self.blocks:
            x = block(x, src_mask=causal_mask, is_causal=True)
        
        # Output
        x = self.ln_f(x)
        logits = self.head(x)  # [B, T, vocab_size]
        return logits

# Smoke test
def smoke_test():
    print("🔥 GPT Smoke Test")
    
    # Setup
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    vocab_size = 100
    model = TinyGPT(vocab_size=vocab_size).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
    
    print(f"Device: {device}")
    print(f"Params: {sum(p.numel() for p in model.parameters()):,}")
    
    # Fake data: learn to copy input shifted by 1
    # Input:  [1, 2, 3, 4, 5]
    # Target: [2, 3, 4, 5, 6]
    def make_batch(batch_size=4, seq_len=16):
        x = torch.randint(0, vocab_size-1, (batch_size, seq_len), device=device)
        y = x + 1  # target is input shifted by 1 in vocab
        return x, y
    
    # Train for 100 steps
    print("\n📚 Training...")
    losses = []
    for step in range(100):
        x, y = make_batch()
        
        # Forward
        logits = model(x)
        loss = F.cross_entropy(logits.view(-1, vocab_size), y.view(-1))
        
        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item())
        if step % 20 == 0:
            print(f"Step {step:3d} | Loss: {loss.item():.4f}")
    
    # Test: can it learn the pattern?
    print("\n🧪 Testing...")
    model.eval()
    with torch.no_grad():
        test_x = torch.tensor([[1, 2, 3, 4, 5]], device=device)
        logits = model(test_x)
        preds = logits.argmax(dim=-1)
        
        print(f"Input:     {test_x[0].tolist()}")
        print(f"Predicted: {preds[0].tolist()}")
        print(f"Expected:  {(test_x[0] + 1).tolist()}")
        
        accuracy = (preds[0] == test_x[0] + 1).float().mean()
        print(f"Accuracy:  {accuracy:.2%}")
    
    # Check overfitting
    final_loss = sum(losses[-10:]) / 10
    print(f"\n✅ Final loss: {final_loss:.4f}")
    
    if final_loss < 0.1:
        print("✅ PASS: Model learned the pattern!")
    else:
        print("❌ FAIL: Model didn't converge")
    
    return final_loss < 0.1

success = smoke_test()